# Saucisse

In [ ]:
# Total system


# H2 production

var_ely_consigne = 1        # 0-1
var_ely_p_kw = 1300
var_ely_v_cell = 2.7
time_total_sim_sec = 3600
choice_h2_prod_fixe = False


if choice_h2_prod_fixe == True :                        
    var_ely_qh2_kgh = int(input("Débit ELY kgH2/h : "))
    var_ely_pressure_bar = int(input("Pressure ELY H2 : "))
elif choice_h2_prod_fixe == False :
    var_ely_qh2_kgh = var_ely_p_kw*pow(10,3) / (2*96485*var_ely_v_cell) * 2.02*3.6    # 2.02 g/mol       0.0899 kg/Nm3       3.6 = s to h & g to kg
    var_ely_pressure_bar = 30

var_ely_qh2_kgh = var_ely_qh2_kgh * var_ely_consigne * time_total_sim_sec/3600
var_ely_e_kwh_total = var_ely_p_kw*pow(10,3) * var_ely_consigne * time_total_sim_sec/3600 / 1000


print("H2 prod (kg/h)           : ", var_ely_qh2_kgh)
print("ELY Energie (kwh)        : ", var_ely_e_kwh_total)
print("H2 ELY pressure (Bar)    : ", var_ely_pressure_bar)



In [ ]:
# H2 production

var_ely_consigne = 1        # 0-1
var_ely_p_kw = 1300
var_ely_v_cell = 2.7
time_total_sim_sec = 3600
choice_h2_prod_fixe = True


if choice_h2_prod_fixe == True :                        
    var_ely_qh2_kgh = int(input("Débit ELY kgH2/h : "))
    var_ely_pressure_bar = int(input("Pressure ELY H2 : "))
elif choice_h2_prod_fixe == False :
    var_ely_qh2_kgh = var_ely_p_kw*pow(10,3) / (2*96485*var_ely_v_cell) * 2.02*3.6    # 2.02 g/mol       0.0899 kg/Nm3       3.6 = s to h & g to kg
    var_ely_pressure_bar = 30

var_ely_qh2_kgh = var_ely_qh2_kgh * var_ely_consigne * time_total_sim_sec/3600
var_ely_e_kwh_total = var_ely_p_kw*pow(10,3) * var_ely_consigne * time_total_sim_sec/3600 / 1000


print("H2 prod (kg/h)           : ", var_ely_qh2_kgh)
print("ELY Energie (kwh)        : ", var_ely_e_kwh_total)
print("H2 ELY pressure (Bar)    : ", var_ely_pressure_bar)

In [24]:
# Stockage H2
import CoolProp


var_stk_masse_kg = 0        # init
var_stk_in_q_kgh = 18
valve_opened = False

time_total_sim_sec = 3600*2.7
time_tick_sim_sec = 0

var_stk_temp_kelv = 273.14+20   # Température ambiante
var_stk_p_bar = 1               # 1 atm
var_stk_v_m3 = 20

var_stk_roh_kgm3 = CoolProp.CoolProp.PropsSI('D','T',var_stk_temp_kelv,'P',var_stk_p_bar*pow(10,5),'hydrogen')
var_stk_u_eint = CoolProp.CoolProp.PropsSI('U','T',var_stk_temp_kelv,'P',var_stk_p_bar*pow(10,5),'hydrogen')
var_stk_h_ent = CoolProp.CoolProp.PropsSI('H','T',var_stk_temp_kelv,'P',var_stk_p_bar*pow(10,5),'hydrogen')
var_stk_y_dan = var_stk_masse_kg*var_stk_u_eint
var_stk_h_ent = -var_stk_u_eint

# Main simulation
while time_tick_sim_sec < time_total_sim_sec :
    time_tick_sim_sec = time_tick_sim_sec + 1

    if valve_opened == False :
        var_stk_out_q_kgh = 0
    elif valve_opened == True :
        var_stk_out_q_kgh = 10       # Faire varier en fct de la masse

    var_stk_masse_kg = var_stk_masse_kg + var_stk_in_q_kgh/3600 - var_stk_out_q_kgh/3600    # Débit to seconde
    var_stk_y_dan = var_stk_y_dan + var_stk_out_q_kgh/3600*var_stk_h_ent - var_stk_in_q_kgh/3600*var_stk_h_ent
    var_stk_u_eint = var_stk_y_dan / var_stk_masse_kg
    var_stk_roh_kgm3 = var_stk_masse_kg / var_stk_v_m3

    var_stk_p_bar = CoolProp.CoolProp.PropsSI('P','T',var_stk_temp_kelv,'D',var_stk_roh_kgm3,'hydrogen')/pow(10,5)
    var_stk_h_ent = CoolProp.CoolProp.PropsSI('H','P',var_stk_p_bar*pow(10,5),'T',var_stk_temp_kelv,'hydrogen')

    var_stk_temp_kelv = CoolProp.CoolProp.PropsSI('T','U',var_stk_u_eint,'D',var_stk_roh_kgm3,'hydrogen')

    var_stk_h_ent = -var_stk_u_eint

print("end")
print("Final masse (kg) : ", var_stk_masse_kg)
print("Final temp (K) : ", var_stk_temp_kelv)
print("Final pressure (Bar) : ", var_stk_p_bar)



end
Final masse (kg) :  48.60000000000641
Final temp (K) :  293.97899973781495
Final pressure (Bar) :  29.99027264533524


# Simu disponibilité

In [15]:
# 2 ELY parallelle 96% dispo chacun
import random

def sim_a_year() :
    days = 0
    panne_counter = 0

    while days < 365 :
        days = days + 1

        ely_1_status = random.randint(0,100)
        ely_2_status = random.randint(0,100)

        if ely_1_status > 96 and ely_2_status > 96 :
            panne_counter = panne_counter + 1
    return panne_counter


years = 0
total_pannes = 0
echantillon = 10000

while years < echantillon :
    years = years + 1
    total_pannes = total_pannes + sim_a_year()

print("Jour indisponible sur {} ans             : ".format(echantillon),total_pannes)
print("Moyenne jour indisponible sur 1 an       : ",total_pannes/echantillon)
print("Disponibilité de 2 stations parallelle   : ",(1-total_pannes / (365*echantillon))*100)

Jour indisponible sur 10000 ans             :  35528
Moyenne jour indisponible sur 1 an       :  3.5528
Disponibilité de 2 stations parallelle   :  99.0266301369863


In [14]:
# 3 ELY parallelle 96% dispo chacun mais besoin de 2 mini
import random

def sim_a_year() :
    days = 0
    panne_counter = 0

    while days < 365 :
        days = days + 1

        ely_1_status = random.randint(0,100)
        ely_2_status = random.randint(0,100)
        ely_3_status = random.randint(0,100)

        if ely_1_status > 96 and ely_2_status > 96 or ely_1_status > 96 and ely_3_status > 96 or ely_2_status > 96 and ely_3_status > 96 :
            panne_counter = panne_counter + 1
    return panne_counter


years = 0
total_pannes = 0
echantillon = 10000

while years < echantillon :
    years = years + 1
    total_pannes = total_pannes + sim_a_year()

print("Jour indisponible sur {} ans             : ".format(echantillon),total_pannes)
print("Moyenne jour indisponible sur 1 an       : ",total_pannes/echantillon)
print("Disponibilité de 2 stations parallelle   : ",(1-total_pannes / (365*echantillon))*100)

Jour indisponible sur 10000 ans             :  16814
Moyenne jour indisponible sur 1 an       :  1.6814
Disponibilité de 2 stations parallelle   :  99.53934246575342
